In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from climbing_wire.homography.homography import compute_homography
from climbing_wire.homography.homography import perspective_transform
from climbing_wire.landmark.compute import PoseImg
from climbing_wire.landmark.compute import compute_landmarks
from climbing_wire.landmark.drawing import draw_landmarks
from climbing_wire.landmark.landmark_list import LandmarkListImg
from climbing_wire.utils.cv import cv_imshow
from climbing_wire.utils.data import get_package_fol
from climbing_wire.utils.mediapipe import JOINT_NAMES
from climbing_wire.utils.mediapipe import get_default_pose_connections
from climbing_wire.video.load import iterate_video_frames, pairwise_video_frames
from matplotlib import pyplot as plt
from pathlib import Path
from typing import List, Tuple, cast
import cv2 as cv
import math
import matplotlib.pyplot as plt
import mediapipe as mp
import mediapipe.python.solutions.drawing_styles as mp_drawing_styles
import mediapipe.python.solutions.drawing_utils as mp_drawing
import mediapipe.python.solutions.pose as mp_pose
import numpy as np
import pathlib
import sys


In [ ]:
# pose_connections = get_default_pose_connections()


In [ ]:
in_fol = Path("/mnt/c/Users/nobilip/Videos/dataset/rock")
in_fn = "PXL_20220901_202119640.mp4"
in_vid_path = in_fol / in_fn


In [ ]:
img_fol = get_package_fol("sample_square")
img_path = img_fol / "photo_0_s.jpg"
img = cv.imread(str(img_path))
# cv_imshow(img)


In [ ]:
landlist: LandmarkListImg = PoseImg()(img)  # type: ignore # not a None
print(f"{landlist=}")
landlist.get_landmark_for_joint("left_foot")


In [ ]:
# https://stackoverflow.com/a/22394181/2237151
# arr = np.empty((0,3), int)
# arr = np.append(arr, np.array([[1,2,3]]), axis=0)


In [ ]:
annotated_image = img.copy()
# draw_landmarks(annotated_image, landlist, pose_connections)
draw_landmarks(annotated_image, landlist)
# cv_imshow(annotated_image)


In [ ]:
landmarks = {}
for jn in JOINT_NAMES:
    landmarks[jn] = np.empty((0, 2), float)


In [ ]:
landmarks

In [ ]:
for jn in JOINT_NAMES:
    lm = landlist.get_landmark_for_joint(jn)
    if lm is None:
        continue
    print(f"{jn} {lm=} {lm.shape=}")
    landmarks[jn] = np.append(landmarks[jn], lm, axis=0)


### Also compute the homography now

In [ ]:
img_path = img_fol / "photo_0_s.jpg"
frame1 = cv.imread(str(img_path))
img_path = img_fol / "photo_1_s.jpg"
frame2 = cv.imread(str(img_path))


In [ ]:
fig, axes = plt.subplots(1, 2)
cv_imshow(frame1, ax=axes[0])
cv_imshow(frame2, ax=axes[1])


In [ ]:
pi = PoseImg(visibility_threshold=0.35)


In [ ]:
landlist1: LandmarkListImg = pi(frame1)  # type: ignore # not a None
landlist2: LandmarkListImg = pi(frame2)  # type: ignore # not a None


In [ ]:
annotated_frame1 = frame1.copy()
draw_landmarks(annotated_frame1, landlist1)
annotated_frame2 = frame2.copy()
draw_landmarks(annotated_frame2, landlist2)

fig, axes = plt.subplots(1, 2)
cv_imshow(annotated_frame1, ax=axes[0])
cv_imshow(annotated_frame2, ax=axes[1])


In [ ]:
M = compute_homography(frame1, frame2)

frame1_warped = cv.warpPerspective(frame1, M, frame1.shape[:2])

fig, axes = plt.subplots(1, 2)
cv_imshow(frame1_warped, ax=axes[0])
cv_imshow(frame2, ax=axes[1])


In [ ]:
landlist1_warped: LandmarkListImg = pi(frame1)  # type: ignore # not a None
landlist1_warped.landmarks_img = perspective_transform(
    landlist1_warped.landmarks_img.astype(np.float64), M
).astype(int)

annotated_frame2 = frame2.copy()
draw_landmarks(annotated_frame2, landlist1_warped)
draw_landmarks(annotated_frame2, landlist2)

fig, axes = plt.subplots(1, 1)
cv_imshow(annotated_frame2, ax=axes)
